In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf


In [2]:
raw_data = pd.read_csv("mnist_train.csv")

In [3]:
raw_data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
class_labels = raw_data['label']

In [5]:
class_labels

0        1
1        0
2        1
3        4
4        0
5        0
6        7
7        3
8        5
9        3
10       8
11       9
12       1
13       3
14       3
15       1
16       2
17       0
18       7
19       5
20       8
21       6
22       2
23       0
24       2
25       3
26       6
27       9
28       9
29       7
        ..
41970    2
41971    3
41972    4
41973    4
41974    3
41975    9
41976    2
41977    4
41978    4
41979    4
41980    7
41981    2
41982    8
41983    7
41984    3
41985    3
41986    0
41987    5
41988    0
41989    5
41990    3
41991    1
41992    9
41993    6
41994    4
41995    0
41996    1
41997    7
41998    6
41999    9
Name: label, Length: 42000, dtype: int64

In [6]:
one_hot_encoded = np.eye(10,10)[class_labels]

In [7]:
one_hot_encoded

array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [8]:
raw_data.drop(['label'],axis=1,inplace=True)

In [9]:
raw_data.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
batch_size = 128

In [11]:
import tensorflow as tf

In [12]:
def down_sample_by_half(input_to_maxpooling):
  
  max_pool_out = tf.nn.max_pool2d(input=input_to_maxpooling,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
  
  return max_pool_out

In [13]:
def variable_creation(shape_of_kernels):
  
  initial_values = tf.random_normal(dtype=tf.float32,shape=shape_of_kernels)
  
  layer_thetas = tf.Variable(initial_values)
  
  return layer_thetas

In [14]:
def convolution_layer(layer_input,shape_of_kernels):
  
  layer_kernels = variable_creation(shape_of_kernels)
  
  print("done")
  
  
  intermediate_output = tf.nn.conv2d(input=layer_input,filter=layer_kernels,strides=[1,1,1,1],padding='SAME')
  
  print("done")
  
  bias_terms = tf.constant(value=0.1,dtype=tf.float32,shape=[shape_of_kernels[3]])
  
  print("done")
  
  final_output_of_conv = intermediate_output + bias_terms
  
  print("done")
  
  max_pooled_output = down_sample_by_half(final_output_of_conv)
 
  print("done")
  
  return max_pooled_output

In [15]:
def fc_layer(layer_input,number_of_inputs,number_of_outputs):
  
  thetas = tf.Variable(initial_value=tf.random_normal(dtype=tf.float32,shape=[number_of_inputs,number_of_outputs]))
  
  dot_product = tf.matmul(layer_input,thetas)
  
  return dot_product

In [16]:
stretched_images = tf.placeholder(dtype=tf.float32,shape=[None,784])

In [17]:
reshaped_images = tf.reshape(stretched_images,shape=[-1,28,28,1])

In [18]:
first_conv_out = convolution_layer(reshaped_images,[5,5,1,32])

done
done
done
done
done


In [19]:
flattened_output = tf.reshape(first_conv_out,shape=[-1,14*14*32])

In [20]:
fc_layer_output = tf.nn.relu(fc_layer(flattened_output,14*14*32,14*14*32))

In [21]:
softmax_layer_logits = tf.nn.softmax(fc_layer(fc_layer_output,14*14*32,10))

In [22]:
batch_labels = tf.placeholder(dtype=tf.float32,shape=[None,10])

In [23]:
cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=softmax_layer_logits,labels=batch_labels))

W0801 09:48:57.761178 18824 deprecation.py:323] From <ipython-input-23-1b5872e4e7fe>:1: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [24]:
opt_obj = tf.train.AdamOptimizer()

In [25]:
optimization_step = opt_obj.minimize(cross_entropy_loss)

In [ ]:
with tf.Session() as S:
  
  S.run(tf.global_variables_initializer())
  
  epochs = 50
  
  for i in range(epochs):
    
    random_indices = np.random.randint(0,raw_data.shape[0],size=batch_size)
    
    images_batch = raw_data.iloc[random_indices]
    
    labels_batch = one_hot_encoded[random_indices]
    
    S.run([optimization_step],feed_dict={stretched_images:images_batch,batch_labels:labels_batch})
    
    loss_fn_value = S.run([cross_entropy_loss],feed_dict={stretched_images:images_batch,batch_labels:labels_batch})
    
    print("Iteration number =",i,"and Loss =",loss_fn_value)